In [131]:
import simpeg.electromagnetics.time_domain as tdem
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import mu_0

from geoana.em.tdem import simple_functions

from ipywidgets import interact, IntSlider, FloatSlider, FloatLogSlider

In [146]:
h = [10, 10]
background_cond = 0.001
layer_cond = 0.1
sigma = [background_cond, layer_cond, background_cond]

ts = np.logspace(-6, -1, 256)

rx = tdem.receivers.PointMagneticFluxTimeDerivative(locations=[0, 0, 0], times=ts)

#waveform = tdem.sources.RampOffWaveform(2.5E-6)
waveform = tdem.sources.StepOffWaveform()
src = tdem.sources.CircularLoop(rx, location=[0, 0, 0], radius=35, waveform=waveform)

srv = tdem.Survey(src)

sim = tdem.Simulation1DLayered(srv, thicknesses=h, sigma=sigma)

In [147]:
def app_cond_late_time(v, t, moment):
    temp = v**2 * t**5 / (moment**2 * mu_0**5) * 20**2
    return temp**(1/3) * np.pi

def do_bz2_plot(top_cond, layer_cond, back_cond, top_layer_thick, middle_layer_thick):

    ts = sim.survey.source_list[0].receiver_list[0].times
    
    sim.sigma = [top_cond, layer_cond, back_cond]
    sim.thicknesses = [top_layer_thick, middle_layer_thick]
    v_sim = -sim.dpred(None)
    v_half_top = -simple_functions.vertical_magnetic_flux_time_deriv_horizontal_loop(
        ts, sigma=top_cond, radius=src.radius
    )
    v_half_bot = -simple_functions.vertical_magnetic_flux_time_deriv_horizontal_loop(
        ts, sigma=back_cond, radius=src.radius
    )
    app_cond = app_cond_late_time(v_sim, ts, src.moment)
    
    fig, axs = plt.subplots(1, 2, sharex=True)
    ax = axs[0]
    ax.loglog(ts, v_sim)
    ax.loglog(ts, v_half_top)
    ax.loglog(ts, v_half_bot)
    
    ax = axs[1]
    ax.loglog(ts, app_cond)
    ax.axhline(top_cond, color='C1')
    ax.axhline(back_cond, color='C2')


# TDEM Interpretation

* Simple interpretation examples
* Inversion techniques
  * 1D Inversion
  * 2D and 3D Inversion
  * Stitched Inversions


## Layered Earth
* For central loop soundings
* Essentially transitions between the different asymptotic values at late time

In [155]:
gen_sigma_slider = lambda : FloatLogSlider(1E-3, min=-5, max=1)
gen_thick_slider = lambda : FloatLogSlider(1E2, min=0, max=4)
out = interact(
    do_bz2_plot,
    top_cond=gen_sigma_slider(),
    layer_cond=gen_sigma_slider(),
    back_cond=gen_sigma_slider(), 
    top_layer_thick=gen_thick_slider(),
    middle_layer_thick=gen_thick_slider()
);

interactive(children=(FloatLogSlider(value=0.001, description='top_cond', max=1.0, min=-5.0), FloatLogSlider(v…